https://www.kaggle.com/datasets/houcembenmansour/predict-diabetes-based-on-diagnostic-measures?resource=download

In [16]:
import pandas as pd

# Importação da base de dados
df = pd.read_csv('/content/sample_data/diabetes.csv')# Retirada do índice de identificação do paciente
df.drop("patient_number", axis=1, inplace=True)# Troca de tipos das variáveis para demonstração
df['diabetes'] = df['diabetes'].apply(lambda x: 0 if x == 'No diabetes' else 1)
df['gender'] = df['gender'].apply(lambda x: 0 if x == 'female' else 1).astype('int')
df['waist_hip_ratio'] = df['waist_hip_ratio'].apply(lambda x : str(x).replace(",",".")).astype('float')
df['chol_hdl_ratio'] = df['chol_hdl_ratio'].apply(lambda x : str(x).replace(",",".")).astype('float')
df['bmi'] = df['bmi'].apply(lambda x : str(x).replace(",",".")).astype('float')

In [17]:
display(df)

,cholesterol,glucose,hdl_chol,chol_hdl_ratio,age,gender,height,weight,bmi,systolic_bp,diastolic_bp,waist,hip,waist_hip_ratio,diabetes
0,193,77,49,3.9,19,0,61,119,22.5,118,70,32,38,0.84,0
1,146,79,41,3.6,19,0,60,135,26.4,108,58,33,40,0.83,0
2,217,75,54,4.0,20,0,67,187,29.3,110,72,40,45,0.89,0
3,226,97,70,3.2,20,0,64,114,19.6,122,64,31,39,0.79,0
4,164,91,67,2.4,20,0,70,141,20.2,122,86,32,39,0.82,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,227,105,44,5.2,83,0,59,125,25.2,150,90,35,40,0.88,0
386,226,279,52,4.3,84,0,60,192,37.5,144,88,41,48,0.85,1
387,301,90,118,2.6,89,0,61,115,21.7,218,90,31,41,0.76,0
388,232,184,114,2.0,91,0,61,127,24.0,170,82,35,38,0.92,1


In [4]:
!pip install pandas-profiling

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
from pandas_profiling import ProfileReport

profile = ProfileReport(df)
profile

profile.to_file("Relatorio.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
df.drop("hip", axis=1, inplace=True)
df.drop("height", axis=1, inplace=True)
df.drop("waist_hip_ratio", axis=1, inplace=True)
df.drop("gender", axis=1, inplace=True)
df.drop("chol_hdl_ratio", axis=1, inplace=True)

In [7]:
# Renomeando as colunas para português 
df.rename(columns=
{"cholesterol": "colesterol_ruim","hdl_chol": "colesterol_bom","age": "idade","weight": "peso","bmi":"imc","systolic_bp": "pressão_sistolica","diastolic_bp": "pressão_diastolica","waist": "cintura"}, 
inplace=True)

Criando o modelo

In [8]:
!pip install pyjanitor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.4/158.4 KB 4.7 MB/s eta 0:00:00


In [9]:
import janitor as jn

from sklearn.model_selection import train_test_split

X,y = jn.get_features_targets(df, target_columns = "diabetes")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state =0)

/usr/local/lib/python3.9/dist-packages/janitor/utils.py:324: FutureWarning: get_features_targets() has moved. Please use ml.get_features_targets().
  warn(message, FutureWarning)
/usr/local/lib/python3.9/dist-packages/janitor/utils.py:352: DeprecationWarning: target_columns is deprecated; use target_column_names
  warn(


In [10]:
!pip install -U scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold

for model in [DummyClassifier, LogisticRegression, DecisionTreeClassifier, KNeighborsClassifier, GaussianNB, SVC, RandomForestClassifier, XGBClassifier]:
  cls = model()
  kfold = KFold(n_splits = 6, random_state = 0, shuffle=True)
  s = cross_val_score(cls,X, y, scoring="roc_auc", cv=kfold)
  print(f"{model.__name__} AUC: {s.mean():.3f} | STD: {s.std():.2f}")

DummyClassifier AUC: 0.500 | STD: 0.00
LogisticRegression AUC: 0.919 | STD: 0.05


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

DecisionTreeClassifier AUC: 0.748 | STD: 0.07
KNeighborsClassifier AUC: 0.903 | STD: 0.04
GaussianNB AUC: 0.929 | STD: 0.04
SVC AUC: 0.940 | STD: 0.04
RandomForestClassifier AUC: 0.941 | STD: 0.03
XGBClassifier AUC: 0.929 | STD: 0.04


In [12]:
from sklearn.metrics import precision_score

#RandomForest
rdf = RandomForestClassifier(random_state=0,n_estimators=200, max_features="auto")
rdf.fit(X_train, y_train)
score = rdf.score(X_test, y_test).item()
print("Random Forest")
print(f'Score: {score:.3f}')
precision = precision_score(y_test, rdf.predict(X_test)).item() *100
print(f'Precisão: {precision}%')

/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Random Forest
Score: 0.923
Precisão: 90.0%


In [13]:
#XGBoost
xgb = XGBClassifier(n_estimators = 20, learning_rate=0.1, n_jobs=5)
xgb.fit(X_train, y_train)
score = xgb.score(X_test, y_test).item()
print("XGBoost")
print(f'Score: {score:.3f}')
precision = precision_score(y_test, xgb.predict(X_test)).item() *100
print(f'Precisão: {precision:.2f}%')

XGBoost
Score: 0.936
Precisão: 90.91%


In [20]:
# Teste com os dois modelos com entradas criadas pelo autor
df_teste_modelo = pd.DataFrame.from_dict([
{'colesterol_ruim':193, 'glucose':400, 'colesterol_bom':49, 'idade':19, 'peso':119, 'imc':22.5,'pressão_sistolica':118, 'pressão_diastolica':70, 'cintura':32},
{'colesterol_ruim':193, 'glucose':77, 'colesterol_bom':49, 'idade':19, 'peso':119, 'imc':22.5,'pressão_sistolica':118, 'pressão_diastolica':70, 'cintura':32}])

pred_test_xgb = xgb.predict(df_teste_modelo)
pred_test_rdf = rdf.predict(df_teste_modelo)
print(f"Resultado Teste RandomForest:{pred_test_rdf}")
print(f"Resultado Teste XGBoost:{pred_test_xgb}")

Resultado Teste RandomForest:[0 0]
Resultado Teste XGBoost:[1 0]
